In [ ]:
%load_ext autoreload
%autoreload 2

from fibsem import utils, acquire, milling, conversions, structures

from fibsem.structures import BeamType, Point, FibsemRectangle


In [ ]:
microscope, settings = utils.setup_session(config_path=r"C:\Users\Admin\Github\autolamella\autolamella")

In [ ]:

settings.image.beam_type = BeamType.ION
ref_image = acquire.new_image(microscope, settings.image)


In [ ]:




pixelsize = ref_image.metadata.pixel_size.x
fiducial_centre = Point(-50e-6, -10e-6)
fiducial_length = 30e-6 # microscope_settings.protocol["fiducial"]["length"] 
fiducial_area = calculate_fiducial_area(settings, fiducial_centre, fiducial_length, pixelsize)

print(fiducial_area)

settings.image.reduced_area = fiducial_area
image = acquire.new_image(microscope, settings.image)
print(image.metadata.image_settings.reduced_area)


## Minimap
Position Display

In [ ]:
%load_ext autoreload
%autoreload 2

from fibsem import utils, acquire, milling, conversions, structures
from fibsem.structures import FibsemImage, BeamType
import os
from fibsem.imaging import _tile

import matplotlib.pyplot as plt
from autolamella.structures import Experiment

# microscope, settings = utils.setup_session()


In [ ]:

PATH = r"C:\Users\Admin\Github\autolamella\autolamella\log\HANNAH-WAFFLE-26072023"
# EXP_PATH = r"C:\Users\Admin\Github\autolamella\autolamella\log\HANNAH-WAFFLE-26072023\experiment.yaml"
exp = Experiment.load(os.path.join(PATH, "experiment.yaml"))
image = FibsemImage.load(os.path.join(PATH, "overview-image-ion-auto-gamma.tif"))
eb_image = FibsemImage.load(os.path.join(PATH, "overview-electron.tif"))
# ion_image = FibsemImage.load(os.path.join(PATH, "overview-ion-trench.tif"))

plt.imshow(image.data, cmap="gray")
plt.show()

In [ ]:
from autolamella.tools.data import create_history_dataframe, calculate_statistics_dataframe

df = create_history_dataframe(exp)
display(df)

import plotly.express as px

fig = px.bar(df, x="petname", y="duration", color="stage", barmode="group", hover_data=df.columns)
fig.show()

In [ ]:

positions = {
    "MillTrench": [],
    "MillUndercut": []
}
for lamella in exp.positions:
    for state in lamella.history:
        
        if state.stage.name in positions.keys():
            positions[state.stage.name].append(state.microscope_state.absolute_position)
            positions[state.stage.name][-1].name = f"{lamella._petname}"# [{state.stage.name}]"

pprint(positions)

In [ ]:

key = "MillTrench"
fig = _tile._plot_positions(image, positions[key], show=True)
fig.savefig(os.path.join(PATH, f"overview-positions-{key.lower()}.png"), dpi=300, bbox_inches='tight')

In [ ]:
fig = _tile._plot_positions(eb_image, positions["MillUndercut"], show=True)
fig.savefig(os.path.join(PATH, "overview-positions-mill-undercut.png"), dpi=300, bbox_inches='tight')


In [ ]:
i  =0 
_n = f"{i+1:02d}"

In [ ]:
_n